In [70]:
import os
import scrapy
import json
import logging
import pandas as pd
import requests
import sys
import numpy as np
import win32com.client as win32
from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess
from sys import platform

In [2]:
pd.set_option('display.width', 200)

class HQPricing:
    def __init__(self, product, part, defender, gadgets, sentrix):
        self.product = product
        self.part = part
        self.defender = defender
        self.gadgets = gadgets
        self.sentrix = sentrix
        
def findphone(phone):
    if phone[-1:] == '/':
        device = phone[:-1]
    elif phone[-5:] == '.html':
        device = phone[:-5]
    else:
        device = phone
    slash = device.rfind('/', 0, len(device)) + 1
    return device[slash:]
        
def Average(prices):
    lst = []
    for price in prices:
        if price > 0:
            lst.append(price)
    return round(np.mean(lst), 2)

def Median(prices):
    lst = []
    for price in prices:
        if price > 0:
            lst.append(price)
    return round(np.median(lst), 2)

def screenPrices(df, descriptions):
    prices = []
    for desc in descriptions:
        try:
            price = df[df['Description'] == desc]['Price'].iloc[0:1].tolist()[0]
            prices.append(price)
        except:
            prices.append(0)
    average_price = Average(prices)
    return average_price

In [3]:
ddefenders = []
html = requests.get('https://www.mobiledefenders.com/').text
bs = BeautifulSoup(html, 'lxml')
possible_links = bs.find_all('a')
for link in possible_links:
    if link.has_attr('href'):
        if link.attrs['href'][0:4] == 'http':
            defender = link.attrs['href']
        else:
            defender = 'https:' + link.attrs['href']
        if defender.find('apple') > 0:
            ddefenders.append(defender)
        if defender.find('samsung') > 0:
            ddefenders.append(defender)

In [4]:
dgadgets = []
html = requests.get('https://www.injuredgadgets.com/').text
bs = BeautifulSoup(html, 'lxml')
possible_links = bs.find_all('a')
for link in possible_links:
    if link.has_attr('href'):
        gadget = link.attrs['href']
        if gadget.find('apple') > 0:
            dgadgets.append(gadget)
        if gadget.find('samsung') > 0:
            dgadgets.append(gadget)
            
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-7-plus/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-7/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-xs-max/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-8-plus/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-6s-plus/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-6-plus/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-se/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-5c/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-xs/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-x/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-8/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-6s/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-6/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-5s/?p=2')
dgadgets.append('https://www.injuredgadgets.com/apple/iphone/iphone-5/?p=2')

In [5]:
dsentrix = []
html = requests.get('https://www.mobilesentrix.com/').text
bs = BeautifulSoup(html, 'lxml')
possible_links = bs.find_all('a')
for link in possible_links:
    if link.has_attr('href'):
        sentrix = link.attrs['href']
        if sentrix.find('apple') > 0:
            dsentrix.append(sentrix)
        if sentrix.find('samsung') > 0:
            dsentrix.append(sentrix)
            
dsentrix.append('https://www.mobilesentrix.com/lcd-assembly-for-iphone-x-aftermarket-plus-quality')


In [6]:
class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('defender.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item
    
class JsonWriterPipeline2(object):
    def open_spider(self, spider):
        self.file = open('sentrix.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item
    
class JsonWriterPipeline3(object):
    def open_spider(self, spider):
        self.file = open('gadgets.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [7]:
class DefenderSpider(scrapy.Spider):
    name = "quotes"
    start_urls = ddefenders
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'defender.json'                           # Used for pipeline 2
    }
    
    def parse(self, response):
        for defender in response.css('div.details-area'):
             yield {
                 #'URL': defender.xpath("//h1/text()").extract(),
                 'Description': defender.css('a ::text').extract_first(),
                 'Price': defender.xpath(".//span[@class='price']/text()").extract_first()  # price is good
          }
                
class SentrixSpider(scrapy.Spider):
    name = "quotes"
    start_urls = dsentrix
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline2':  1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'sentrix.json'                            # Used for pipeline 2
    }
    
    def parse(self, response):
        for sentrix in response.css('div.category-products'):
             yield {
                 #'URL': sentrix.xpath("//h1/text()").extract(),
                 'Description': sentrix.xpath(".//span[@class='product-name']/text()").extract(),  
                 'Price': sentrix.xpath(".//span[@class='price']/text()").extract() 
         }
                
class GadgetsSpider(scrapy.Spider):
    name = "quotes"
    start_urls = dgadgets
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline3':  1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'gadgets.json'                            # Used for pipeline 2
    }         
            
    def parse(self, response):
        for gadget in response.css('div.product'):
            yield {
                #'URL': gadget.xpath('//div[@class="breadcrumbs"]').extract(),
                'Desc': gadget.css('a ::text').extract_first(),
                'Price': gadget.xpath(".//span[@class='price']/text()").extract_first() } 

In [8]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(DefenderSpider)
process.crawl(SentrixSpider)
process.crawl(GadgetsSpider)
process.start()

2019-02-13 11:00:44 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-02-13 11:00:44 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 18.9.0, Python 3.6.7 |Anaconda, Inc.| (default, Dec 10 2018, 20:35:02) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2p  14 Aug 2018), cryptography 2.5, Platform Windows-10-10.0.17134-SP0
2019-02-13 11:00:44 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'defender.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [9]:
frames = []
df_sentrix = pd.read_json('sentrix.jl', lines = True) 

In [10]:
for index, row in df_sentrix.iterrows():
    df = pd.DataFrame([row[0], row[1]]) 
    df = df.T
    frames.append(df)

df_sentrix = pd.concat(frames)
df_sentrix.drop_duplicates(inplace = True)
df_sentrix.reset_index(drop = True, inplace = True)
df_sentrix['Company'] = 'Mobile Sentrix'
df_sentrix.columns = ['Description', 'Price', 'Company']

In [11]:
for index, row in df_sentrix.iterrows():
    df_sentrix.at[index, 'Price'] = str(row['Price']).strip('\n')

In [12]:
df_defender = pd.read_json('defender.jl', lines=True)   
df_defender['Company'] = 'Mobile Defenders'
df_defender.columns = ['Description', 'Price', 'Company']

In [13]:
df_gadgets = pd.read_json('gadgets.jl', lines = True)
df_gadgets['Company'] = 'Injured Gadgets'
df_gadgets.columns = ['Description', 'Price', 'Company']

for index, row in df_gadgets.iterrows():
    df_gadgets.at[index, 'Description'] = str(row['Description']).strip('\n').strip()
    df_gadgets.at[index, 'Price'] = str(row['Price']).strip('$').strip()

df_gadgets.replace('None', '', inplace = True)
df_gadgets = df_gadgets[df_gadgets["Description"] != ""]
df_gadgets = df_gadgets[df_gadgets['Price'] != '']
df_gadgets = df_gadgets[df_gadgets['Price'] != None]
df_gadgets.drop_duplicates(inplace = True)
df_gadgets.reset_index(drop = True, inplace = True)

In [14]:
for index, row in df_defender.iterrows():
    df_defender.at[index, 'Description'] = str(row['Description']).strip()
    df_defender.at[index, 'Price'] = float(str(row['Price']).strip().strip('$'))
    
for index, row in df_sentrix.iterrows():
    df_sentrix.at[index, 'Description'] = str(row['Description']).strip()
    df_sentrix.at[index, 'Price'] = float(str(row['Price']).strip().strip('$'))
    
for index, row in df_gadgets.iterrows():
    df_gadgets.at[index, 'Description'] = str(row['Description']).strip()
    df_gadgets.at[index, 'Price'] = float(str(row['Price']).strip().strip('$'))

In [15]:
df_defender[df_defender['Description'] == 'Battery for iPhone 8 (SELECT)']['Price'].iloc[0:1].tolist()[0]

16.99

In [16]:
# iPhoneX
iPhoneX = HQPricing('iPhone X', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone X']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for the iPhone X']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone X (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])
#iPhoneSE
iPhoneSE = HQPricing('iPhone SE', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone SE']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for the iPhone SE']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone SE (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 8
iPhone8 = HQPricing('iPhone 8', 'Battery', 
                   df_defender[df_defender['Description'] == 'Battery for iPhone 8 (SELECT)']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for the iPhone 8 (4.7")']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone 8 (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 7
iPhone7 = HQPricing('iPhone 7', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 7 (SELECT)']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for the iPhone 7 (4.7")']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone 7 (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 7 Plus
iPhone7Plus = HQPricing('iPhone 7 Plus', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 7 Plus (SELECT)']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for iPhone 7 Plus (5.5")']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone 7 Plus (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 6
iPhone6 = HQPricing('iPhone 6', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 6 (SELECT)']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for the iPhone 6 (4.7")']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery for iPhone 6 (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 6 Plus
iPhone6Plus = HQPricing('iPhone 6 Plus', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 6 Plus (SELECT)']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for iPhone 6 Plus (5.5")']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery for iPhone 6 Plus (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone5 
iPhone5 = HQPricing('iPhone 5', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 5']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for iPhone 5']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery for iPhone 5 (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 5C
iPhone5C = HQPricing('iPhone 5C', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 5C']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for iPhone 5C']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery for iPhone 5C (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 5S
iPhone5S = HQPricing('iPhone 5S', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 5S']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for the iPhone 5S']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery for iPhone 5S (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 6S
iPhone6S = HQPricing('iPhone 6S', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 6S (SELECT)']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == '1715 mAh Battery for iPhone 6S']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone 6S (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone 6S Plus
iPhone6SPlus = HQPricing('iPhone 6S Plus', 'Battery', 
                   df_defender[df_defender['Description'] == 'Battery for iPhone 6S Plus (SELECT)']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == '1715 mAh Battery for iPhone 6S']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone 6S Plus (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

#iPhone SE
iPhoneSE = HQPricing('iPhone SE', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone SE']['Price'].iloc[0:1].tolist()[0],
                   df_gadgets[df_gadgets['Description'] == 'Battery for the iPhone SE']['Price'].iloc[0:1].tolist()[0], 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone SE (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])

# #iPhone 8 Plus strangely scrape is not picking up the battery for the iPhone 8 Plus
iPhone8Plus = HQPricing('iPhone 8 Plus', 'Battery', df_defender[df_defender['Description'] == 'Battery for iPhone 8 Plus (SELECT)']['Price'].iloc[0:1].tolist()[0],
                   0, 
                   df_sentrix[df_sentrix['Description'] == 'Replacement Battery For iPhone 8 Plus (Premium Quality, AmpSentrix)']['Price'].iloc[0:1].tolist()[0])



In [17]:
sentrix_descriptions = ['OLED Assembly With Force Touch Panel For iPhone X (Aftermarket Plus Quality, GX)', 
                'LCD Assembly With Force Touch Panel For iPhone X (Aftermarket Quality, Tianma)']

defender_descriptions = ['OLED & Digitizer Frame Assembly for iPhone X (Select) - Black',
                        'LCD & Digitizer Frame Assembly for iPhone X (w/ Force Touch Panel) (Basic) - Black']

gadget_descriptions = ['LCD Version Screen and Digitizer Assembly, Black, For iPhone X (After Market)', 
                       'Hard OLED Screen and Digitizer Assembly, Black, for iPhone X',
                       'Midgrade OLED Screen and Digitizer Assembly, Black, for iPhone X']

# iPhone X
iPhoneX_Screen = HQPricing('iPhone X', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))


2019-02-13 11:03:55 [py.warnings] WARNING: C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

2019-02-13 11:03:55 [py.warnings] WARNING: C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



In [18]:
sentrix_descriptions = ['LCD Assembly With Frame for Samsung Galaxy S9 (Premium) (Midnight Black Frame)',
                        'LCD Assembly With Frame for Samsung Galaxy S9 (Premium) (Titanium Grey Frame)',
                        'LCD Assembly With Frame for Samsung Galaxy S9 (Premium) (Coral Blue Frame)',
                        'LCD Assembly With Frame for Samsung Galaxy S9 (Premium) (Lilac Purple Frame)']

gadget_descriptions = ['Genuine Samsung Galaxy Note 9 LCD Digitizer display Assembly with front housing, Black',
                       'Genuine Samsung Galaxy Note 9 LCD Digitizer display Assembly with front housing, Blue',
                       'Genuine Samsung Galaxy Note 9 LCD Digitizer display Assembly with front housing, Purple']

defender_descriptions = ['LCD & Digitizer Display Assembly (w/Frame) for Samsung Galaxy S9 (Prime - OEM) - Midnight Black',
                         'LCD & Digitizer Display Assembly (w/Frame) for Samsung Galaxy S9 (Prime - OEM) - Lilac Purple',
                         'LCD & Digitizer Display Assembly (w/Frame) for Samsung Galaxy S9 (Prime - OEM) - Coral Blue',
                         'LCD & Digitizer Display Assembly (w/Frame) for Samsung Galaxy S9 (Prime - OEM) - Titanium Gray']

SamsungGS9_Screen = HQPricing('Samsung Galaxy S9', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Replacement Battery For Samsung Galaxy S9']

gadget_descriptions = ['']

defender_descriptions = ['Battery for Samsung Galaxy S9']

SamsungGS9_Battery = HQPricing('Samsung Galaxy S9', 'Battery', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

# ignore error messages

In [19]:
sentrix_descriptions = ['Charging Port Flex Cable For Samsung Galaxy Note 9']

gadget_descriptions = ['USB Type C Charging Dock Port Flex Cable for Samsung Galaxy Note 9 N960U']

defender_descriptions = ['Charging Port Flex Cable for Samsung Galaxy Note 9 (N960V / N960T / N960A / N960P / N960R4)']

# Samsung Note 9
SamsungNote9_charging = HQPricing('Samsung Note 9', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))


sentrix_descriptions = ['LCD Assembly With Frame For Samsung Galaxy Note 9 (Premium) (Lavender Purple Frame)',
                        'LCD Assembly With Frame For Samsung Galaxy Note 9 (Premium) (Midnight Black Frame)',
                        'LCD Assembly With Frame For Samsung Galaxy Note 9 (Premium) (Ocean Blue Frame)']

gadget_descriptions = ['Genuine Samsung Galaxy Note 9 LCD Digitizer display Assembly with front housing, Black',
                       'Genuine Samsung Galaxy Note 9 LCD Digitizer display Assembly with front housing, Blue',
                       'Genuine Samsung Galaxy Note 9 LCD Digitizer display Assembly with front housing, Purple']

defender_descriptions = ['LCD & Digitizer Display Assembly (w/Frame) for Samsung Galaxy Note 9 (Prime - OEM) - Ocean Blue',
                         'LCD & Digitizer Display Assembly (w/Frame) for Samsung Galaxy Note 9 (Prime - OEM) - Lavander Purple',
                         'LCD & Digitizer Display Assembly (w/Frame) for Samsung Galaxy Note 9 (Prime - OEM) - Midnight Black']

# Samsung Note 9
SamsungNote9_Screen = HQPricing('Samsung Note 9', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['']

gadget_descriptions = ['Replacement Battery for Samsung Galaxy Note 9']

defender_descriptions = ['Battery for Galaxy Note 9']

# Samsung Note 9
SamsungNote9_Battery = HQPricing('Samsung Note 9', 'Battery', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))


In [20]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone XS (Premium) (Gold)',
                        'Charging Port Flex Cable For iPhone XS (Premium) (Space Grey)']

defender_descriptions = ['Charging Port Flex Cable for iPhone XS (5.8") - Space Gray',
                         'Charging Port Flex Cable for iPhone XS (5.8") - Silver',
                         'Charging Port Flex Cable for iPhone XS (5.8") - Gold']

gadget_descriptions = ['Replacement Wireless Charging Chip with Flex Cable for the iPhone XS']

# iPhone XS
iPhoneXS_charging = HQPricing('iPhone XS', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))
                                        

sentrix_descriptions = ['OLED Assembly With Force Touch Panel For iPhone XS (OEM Pulls / Grade A)']

defender_descriptions = ['OLED & Digitizer Frame Assembly for iPhone XS (5.8") (Prime) - Black']

gadget_descriptions = ['Premium Quality OLED Screen and Digitizer Assembly, Black, for iPhone XS']

# iPhone XS
iPhoneXS_Screen = HQPricing('iPhone XS', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))


sentrix_descriptions = ['LCD Assembly for iPhone XR (Premium Quality)']

defender_descriptions = ['LCD & Digitizer Assembly for iPhone XR (6.1") (Prime) - Black']

gadget_descriptions = ['Premium Quality LCD Screen and Digitizer Assembly, Black, for iPhone XR']

# iPhone XR
iPhoneXR_Screen = HQPricing('iPhone XR', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Charging Port Flex Cable For iPhone XR (Premium) (White)',
                        'Charging Port Flex Cable For iPhone XR (Premium) (Black)']

defender_descriptions = ['']

gadget_descriptions = ['']

# iPhone XR
iPhoneXR_charging = HQPricing('iPhone XR', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [21]:
sentrix_descriptions = ['LCD Assembly for iPhone 5S/SE (Aftermarket Quality, Shenchao) (White)',
                        'LCD Assembly for iPhone 5S/SE (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly for iPhone 5S / SE (Aftermarket Plus Quality, Tianma) (White)',
                        'LCD Assembly for iPhone 5S / SE (Aftermarket Plus Quality, Tianma) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone SE (Select) - Black',
                        'LCD & Digitizer Frame Assembly for iPhone SE (Select) - White']

gadget_descriptions = ['Midgrade LCD Screen and Digitizer Assembly, White, for iPhone SE', 
                       'Midgrade LCD Screen and Digitizer Assembly, Black, for iPhone SE']

# iPhone SE
iPhoneSE_Screen = HQPricing('iPhone SE', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [22]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 7 Plus (Aftermarket Quality) (Jet Black / Matte Black)',
                        'Charging Port Flex Cable For iPhone 7 Plus (Aftermarket Quality) (Gold / Rose Gold)',
                        'Charging Port Flex Cable For iPhone 7 Plus (Aftermarket Quality) (Silver)']

defender_descriptions = ['Charging Port Flex Cable for iPhone 7 Plus (5.5") - Black',
                        'Charging Port Flex Cable for iPhone 7 Plus (5.5") - White',
                        'Charging Port Flex Cable for iPhone 7 Plus (5.5") - Light Gray']

gadget_descriptions = ['Charging Dock Flex Cable for the iPhone 7 Plus (5.5"), Black',
                      'Charging Dock Flex Cable for the iPhone 7 Plus (5.5"), Grey',
                      'Charging Dock Flex Cable for the iPhone 7 Plus (5.5"), White']

iPhone7Plus_charging = HQPricing('iPhone 7 Plus', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [23]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 7 (Aftermarket Quality) (Jet Black / Matte Black)',
                        'Charging Port Flex Cable For iPhone 7 (Aftermarket Quality) (Gold / Rose Gold)',
                        'Charging Port Flex Cable For iPhone 7 (Aftermarket Quality) (Silver)']

defender_descriptions = ['Charging Port Flex Cable for iPhone 7 (4.7") - Black',
                        'Charging Port Flex Cable for iPhone 7 (4.7") - White',
                        'Charging Port Flex Cable for iPhone 7 (4.7") - Light Gray']

gadget_descriptions = ['Charging Dock Flex Cable for the iPhone 7 (4.7"), Grey',
                      'Charging Dock Flex Cable for the iPhone 7 (4.7"), Black',
                      'Charging Dock Flex Cable for the iPhone 7 (4.7"), White']

iPhone7_charging = HQPricing('iPhone 7', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [24]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 5 (Silver)',
                        'Charging Port Flex Cable For iPhone 5 (Space Grey)']

defender_descriptions = ['Charging Port & Headphone Jack Flex Cable for iPhone 5 - White',
                         'Charging Port & Headphone Jack Flex Cable for iPhone 5 - Black']

gadget_descriptions = ['Lightning Dock/Headphone Jack Connector Flex Cable, Black, for iPhone 5',
                      'Lightning Dock/Headphone Jack Connector Flex Cable, White, for iPhone 5']

iPhone5_charging = HQPricing('iPhone 5', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [25]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 5C (Black)']

defender_descriptions = ['Charging Port & Headphone Jack Flex Cable for iPhone 5C - Black']

gadget_descriptions = ['Charging Dock/Headphone Jack Flex Cable for the iPhone 5C, Black']

iPhone5C_charging = HQPricing('iPhone 5C', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [26]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 5S (Silver)',
                        'Charging Port Flex Cable For iPhone 5S (Space Grey)']

defender_descriptions = ['Charging Port & Headphone Jack Flex Cable for iPhone 5S - Black',
                         'Charging Port & Headphone Jack Flex Cable for iPhone 5S - White']

gadget_descriptions = ['Charging Dock/Headphone Jack Flex Cable for the iPhone 5S, White',
                       'Charging Dock/Headphone Jack Flex Cable for the iPhone 5S, Black']

iPhone5S_charging = HQPricing('iPhone 5S', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [27]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 8 (Aftermarket Quality) (Jet Black / Matte Black)',
                        'Charging Port Flex Cable For iPhone 8 (Aftermarket Quality) (Gold / Rose Gold)',
                        'Charging Port Flex Cable For iPhone 8 (Aftermarket Quality) (Silver)']

defender_descriptions = ['Charging Port Flex Cable for iPhone 8 (4.7") - Space Gray',
                        'Charging Port Flex Cable for iPhone 8 (4.7") - Silver',
                        'Charging Port Flex Cable for iPhone 8 (4.7") - Gold']

gadget_descriptions = ['Charging Dock Flex Cable for the iPhone 8 (4.7"), Grey',
                      'Charging Dock Flex Cable for the iPhone 8 (4.7"), Black',
                      'Charging Dock Flex Cable for the iPhone 8 (4.7"), White']

iPhone8_charging = HQPricing('iPhone 8', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Charging Port Flex Cable For iPhone 8 Plus (Aftermarket Quality) (Jet Black / Matte Black)',
                        'Charging Port Flex Cable For iPhone 8 Plus (Aftermarket Quality) (Gold / Rose Gold)',
                        'Charging Port Flex Cable For iPhone 8 (Aftermarket Quality) (Silver)']

defender_descriptions = ['Charging Port Flex Cable for iPhone 8 Plus (5.5") - Space Gray',
                        'Charging Port Flex Cable for iPhone 8 Plus (5.5") - Silver',
                        'Charging Port Flex Cable for iPhone 8 Plus (5.5") - Gold']

gadget_descriptions = ['Charging Dock Flex Cable for the iPhone 8 Plus (5.5"), Grey',
                      'Charging Dock Flex Cable for the iPhone 8 Plus (5.5"), Black',
                      'Charging Dock Flex Cable for the iPhone 8 Plus (5.5"), White']

iPhone8Plus_charging = HQPricing('iPhone 8 Plus', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [28]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 6S Plus (Aftermarket Quality) (Jet Black / Matte Black)',
                        'Charging Port Flex Cable For iPhone 6S Plus (Aftermarket Quality) (Gold / Rose Gold)',
                        'Charging Port Flex Cable For iPhone 6S Plus (Aftermarket Quality) (Silver)']

defender_descriptions = ['Charging Port & Headphone Jack Flex Cable for iPhone 6S Plus (5.5") - Dark Gray',
                        'Charging Port & Headphone Jack Flex Cable for iPhone 6S Plus (5.5") - Light Gray',
                        'Charging Port & Headphone Jack Flex Cable for iPhone 6S Plus (5.5") - White']

gadget_descriptions = ['Charging Dock Headphone Jack Flex Cable for the iPhone 6S Plus (5.5"), Grey',
                      'Charging Dock Headphone Jack Flex Cable for the iPhone 6S Plus (5.5"), Light Grey',
                      'Charging Dock Headphone Jack Flex Cable for the iPhone 6S Plus (5.5"), White / Gold']

iPhone6SPlus_charging = HQPricing('iPhone 6S Plus', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Charging Port Flex Cable For iPhone 6S (Aftermarket Quality) (Jet Black / Matte Black)',
                        'Charging Port Flex Cable For iPhone 6S (Aftermarket Quality) (Gold / Rose Gold)',
                        'Charging Port Flex Cable For iPhone 6S (Aftermarket Quality) (Silver)']

defender_descriptions = ['Charging Port & Headphone Jack Flex Cable for iPhone 6S (4.7") - Dark Gray',
                        'Charging Port & Headphone Jack Flex Cable for iPhone 6S (4.7") - Light Gray',
                        'Charging Port & Headphone Jack Flex Cable for iPhone 6S (4.7") - White']

gadget_descriptions = ['Charging Dock Headphone Jack Flex Cable for the iPhone 6S (4.7"), White',
                      'Charging Dock Headphone Jack Flex Cable for the iPhone 6S (4.7"), Dark Grey']

iPhone6S_charging = HQPricing('iPhone 6S', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [29]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 6 (Aftermarket Quality) (Gold)',
                        'Charging Port Flex Cable For iPhone 6 (Aftermarket Quality) (Space Grey)',
                        'Charging Port Flex Cable For iPhone 6  (Aftermarket Quality) (Silver)']

defender_descriptions = ['Charging Port & Headphone Jack Flex Cable for iPhone 6 (4.7") - Light Gray',
                         'Charging Port & Headphone Jack Flex Cable for iPhone 6 (4.7") - Dark Gray',
                         'Charging Port & Headphone Jack Flex Cable for iPhone 6 (4.7") - White']

gadget_descriptions = ['Charging Dock/Headphone Jack Flex Cable for the iPhone 6 (4.7"), White',
                       'Charging Dock/Headphone Jack Flex Cable for the iPhone 6 (4.7"), Grey']

iPhone6_charging = HQPricing('iPhone 6', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [30]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone 6 Plus (Aftermarket Quality) (Jet Black / Matte Black)',
                        'Charging Port Flex Cable For iPhone 6 Plus (Aftermarket Quality) (Gold / Rose Gold)',
                        'Charging Port Flex Cable For iPhone 6 Plus (Aftermarket Quality) (Silver)']

defender_descriptions = ['Charging Port & Headphone Jack Flex Cable for iPhone 6 Plus (5.5") - Light Gray',
                         'Charging Port & Headphone Jack Flex Cable for iPhone 6 Plus (5.5") - Dark Gray',
                         'Charging Port & Headphone Jack Flex Cable for iPhone 6 Plus (5.5") - White']

gadget_descriptions = ['Charging Dock/Headphone Jack Flex Cable for the iPhone 6 Plus (5.5"), White',
                       'Charging Dock/Headphone Jack Flex Cable for the iPhone 6 Plus (5.5"), Grey']

iPhone6Plus_charging = HQPricing('iPhone 6 Plus', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [31]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone X (Premium) (Silver)',
                        'Charging Port Flex Cable For iPhone X (Premium) (Space Grey)']

defender_descriptions = ['Charging Port Flex Cable for iPhone X - Space Gray',
                        'Charging Port Flex Cable for iPhone X - Silver']

gadget_descriptions = ['Charging Dock Flex Cable for the iPhone X, Black',
                      'Replacement Charging Dock Flex for the iPhone X, Silver']

iPhoneX_charging = HQPricing('iPhone X', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [32]:
sentrix_descriptions = ['Charging Port Flex Cable For iPhone SE (Silver / Gold / Rose Gold)',
                        'Charging Port Flex Cable For iPhone SE (Space Grey)']

defender_descriptions = ['Charging Port & Headphone Jack Flex Cable for iPhone SE - White',
                         'Charging Port & Headphone Jack Flex Cable for iPhone SE - Black']

gadget_descriptions = ['Charging Dock/Headphone Jack Flex Cable for the iPhone SE, Black',
                       'Charging Dock/Headphone Jack Flex Cable for the iPhone SE, White']

iPhoneSE_charging = HQPricing('iPhone SE', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [33]:
sentrix_descriptions = ['LCD Assembly For iPhone 8 (Aftermarket Quality, AUO) (White)', 
                        'LCD Assembly For iPhone 8 (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly for iPhone 8 (Aftermarket Plus Quality, XO4) (White)',
                        'LCD Assembly for iPhone 8 (Aftermarket Plus Quality, XO4) (White)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 8 (4.7") (Select) - Black',
                         'LCD & Digitizer Frame Assembly for iPhone 8 (4.7") (Select) - White']

gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 8 (4.7")', 
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 8 (4.7")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 8 (4.7")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 8 (4.7")']

# iPhone8
iPhone8_Screen = HQPricing('iPhone 8', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [34]:
sentrix_descriptions = ['LCD Assembly For iPhone 8 Plus (Aftermarket Quality, HuaXing) (White)',
                        'LCD Assembly For iPhone 8 Plus (Aftermarket Quality, HuaXing) (Black)',
                        'LCD Assembly for iPhone 8 Plus (Aftermarket Plus Quality, AUO) (White)',
                        'LCD Assembly for iPhone 8 Plus (Aftermarket Plus Quality, XO4) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 8 Plus (5.5") (Select) - Black', 
                        'LCD & Digitizer Frame Assembly for iPhone 8 Plus (5.5") (Select) - White',
                        'LCD & Digitizer Frame Assembly for iPhone 8 Plus (5.5") (Basic) - Black',
                        'LCD & Digitizer Frame Assembly for iPhone 8 Plus (5.5") (Basic) - White']


gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 8 Plus (5.5")', 
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 8 Plus (5.5")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (AUO LCD), White, for iPhone 8 Plus (5.5")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (AUO LCD), Black, for iPhone 8 Plus (5.5")']

# iPhone8 Plus
iPhone8Plus_Screen = HQPricing('iPhone 8 Plus', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [35]:
sentrix_descriptions = ['LCD Assembly for iPhone 6 (Aftermarket Quality, Shenchao) (White)',
                        'LCD Assembly for iPhone 6 (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly for iPhone 6 (Aftermarket Plus Quality, XO4) (White)',
                        'LCD Assembly for iPhone 6 (Aftermarket Plus Quality, XO4) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 6 (4.7") (Select) - White (Bulk Pricing Available)', 
                        'LCD & Digitizer Frame Assembly for iPhone 6 (4.7") (Basic) - White (Bulk Pricing Available)',
                        'LCD & Digitizer Frame Assembly for iPhone 6 (4.7") (Select) - Black (Bulk Pricing Available)',
                        'LCD & Digitizer Frame Assembly for iPhone 6 (4.7") (Basic) - Black (Bulk Pricing Available)']


gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 6 (4.7")',
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 6 (4.7")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 6 (4.7")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 6 (4.7")']

# iPhone 6
iPhone6_Screen = HQPricing('iPhone 6', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [36]:
sentrix_descriptions = ['LCD Assembly For iPhone 7 (Aftermarket Quality, Shenchao) (White)',
                        'LCD Assembly For iPhone 7 (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly For iPhone 7 (Aftermarket Plus Quality, XO4) (White)',
                        'LCD Assembly For iPhone 7 (Aftermarket Plus Quality, XO4) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 7 (4.7") (Select) - White (Bulk Pricing Available)', 
                         'LCD & Digitizer Frame Assembly for iPhone 7 (4.7") (Basic) - White (Bulk Pricing Available)',
                         'LCD & Digitizer Frame Assembly for iPhone 7 (4.7") (Select) - Black (Bulk Pricing Available)',
                         'LCD & Digitizer Frame Assembly for iPhone 7 (4.7") (Basic) - Black (Bulk Pricing Available)']


gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 7 (4.7")',
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 7 (4.7")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 7 (4.7")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 7 (4.7")']

# iPhone 7
iPhone7_Screen = HQPricing('iPhone 7', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [37]:
sentrix_descriptions = ['LCD Assembly For iPhone 7 Plus (Aftermarket Quality, Shenchao) (White)',
                        'LCD Assembly For iPhone 7 Plus (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly For iPhone 7 Plus (Aftermarket Plus Quality, XO4) (White)',
                        'LCD Assembly For iPhone 7 Plus (Aftermarket Plus Quality, XO4) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 7 Plus (5.5") (Select) - White (Bulk Pricing Available)', 
                         'LCD & Digitizer Frame Assembly for iPhone 7 Plus (5.5") (Basic) - White (Bulk Pricing Available)',
                         'LCD & Digitizer Frame Assembly for iPhone 7 Plus (5.5") (Select) - Black (Bulk Pricing Available)',
                         'LCD & Digitizer Frame Assembly for iPhone 7 Plus (5.5") (Basic) - Black (Bulk Pricing Available)']


gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 7 Plus (5.5")',
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 7 Plus (5.5")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 7 Plus (5.5")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 7 Plus (5.5")']

# iPhone 7 Plus
iPhone7Plus_Screen = HQPricing('iPhone 7 Plus', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [38]:
sentrix_descriptions = ['LCD Assembly for iPhone 5 (Aftermarket Quality, Shenchao) (White)',
                        'LCD Assembly for iPhone 5 (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly for iPhone 5 (Aftermarket Plus Quality, Tianma) (White)',
                        'LCD Assembly for iPhone 5 (Aftermarket Plus Quality, Tianma) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 5 (Select) - Black', 
                         'LCD & Digitizer Frame Assembly for iPhone 5 (Select) - White']

gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 5',
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 5',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 5',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 5']

# iPhone 5
iPhone5_Screen = HQPricing('iPhone 5', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [39]:
sentrix_descriptions = ['Home Button For iPhone 5 (White)',
                        'Home Button For iPhone 5 / 5C (Black)']

defender_descriptions = ['Home Button (w/ Rubber Gasket) for iPhone 5 / iPhone 5C (Generic) - Black']

gadget_descriptions = ['Home Button for iPhone 5 & 5C (White)',
                       'Home Button for iPhone 5 & 5C (Black)']

# iPhone 5
iPhone5_Home = HQPricing('iPhone 5', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [40]:
sentrix_descriptions = ['Home Button With Flex For iPhone 6S / iPhone 6S Plus (Gold)',
                        'Home Button With Flex For iPhone 6S / iPhone 6S Plus (Silver)',
                        'Home Button With Flex For iPhone 6S / iPhone 6S Plus (Rose Gold)',
                        'Home Button With Flex For iPhone 6S / iPhone 6S Plus (Space Grey)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6S Plus (5.5") - Black (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6S Plus (5.5") - White (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6S Plus (5.5") - Gold (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6S Plus (5.5") - Rose Gold (Fingerprint scanner is aftermarket - Biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPhone 6S (4.7") & 6S Plus (5.5"), Black',
                       'Home Button Flex Cable for the iPhone 6S (4.7") & 6S Plus (5.5"), Silver',
                       'Home Button Flex Cable for the iPhone 6S (4.7") & 6S Plus (5.5"), Gold',
                       'Home Button Flex Cable for the iPhone 6S (4.7") & 6S Plus (5.5"), Rose Gold']

# iPhone 6S Plus
iPhone6SPlus_Home = HQPricing('iPhone 6S Plus', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [41]:
sentrix_descriptions = ['Home Button Extension Flex For iPhone 6 Plus',
                        'Home Button With Flex For iPhone 6 / 6 Plus (Gold)',
                        'Home Button With Flex For iPhone 6 / 6 Plus (Silver)',
                        'Home Button With Flex For iPhone 6 / 6 Plus (Space Grey)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6 Plus (5.5") - Black (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6 Plus (5.5") - White (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6 Plus (5.5") - Gold (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6 Plus (5.5") - Rose Gold (Fingerprint scanner is aftermarket - Biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPhone 6 Plus (5.5"), Grey',
                       'Home Button Flex Cable for the iPhone 6 Plus (5.5"), Silver',
                       'Home Button Flex Cable for the iPhone 6 Plus (5.5"), Gold']

# iPhone 6 Plus
iPhone6Plus_Home = HQPricing('iPhone 6 Plus', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [42]:
sentrix_descriptions = ['Home Button For iPhone 5 / 5C (Black)',
                        'Home Button Flex for Iphone 5C']

defender_descriptions = ['Home Button Flex Cable for iPhone 5C']

gadget_descriptions = ['Home Button for iPhone 5 & 5C (Black)',
                       'Home Button for iPhone 5 & 5C (White)']

# iPhone 5C
iPhone5C_Home = HQPricing('iPhone 5C', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [43]:
sentrix_descriptions = ['Home Button With Flex For iPhone 5S / SE (Gold)',
                        'Home Button With Flex For iPhone 5S / SE (Silver)'
                        'Home Button With Flex For iPhone 5S / SE (Space Grey)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 5S / iPhone SE - Black (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 5S / iPhone SE - Gold (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 5S / iPhone SE - White (Fingerprint scanner is aftermarket - Biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPhone 5S, Black',
                       'Home Button Flex Cable for the iPhone 5S, Silver',
                       'Home Button Flex Cable for the iPhone 5S, Gold']

# iPhone 5S
iPhone5S_Home = HQPricing('iPhone 5S', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [44]:
sentrix_descriptions = ['Home Button With Flex For iPhone 6 / 6 Plus (Gold)',
                        'Home Button With Flex For iPhone 6 / 6 Plus (Silver)',
                        'Home Button With Flex For iPhone 6 / 6 Plus (Space Grey)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6 (4.7") - Black (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6 (4.7") - White (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6 (4.7") - Gold (Fingerprint scanner is aftermarket - Biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPhone 6 (4.7"), Grey',
                       'Home Button Flex Cable for the iPhone 6 (4.7"), Silver',
                       'Home Button Flex Cable for the iPhone 6 (4.7"), Gold']

# iPhone 6
iPhone6_Home = HQPricing('iPhone 6', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [45]:
sentrix_descriptions = ['Home Button With Flex For iPhone 6S / iPhone 6S Plus (Gold)',
                        'Home Button With Flex For iPhone 6S / iPhone 6S Plus (Rose Gold)'
                        'Home Button With Flex For iPhone 6S / iPhone 6S Plus (Silver)',
                        'Home Button With Flex For iPhone 6S / iPhone 6S Plus (Space Grey)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6S Plus (5.5") - Rose Gold (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6S Plus (5.5") - White (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6S Plus (5.5") - Gold (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 6S Plus (5.5") - Black (Fingerprint scanner is aftermarket - Biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPhone 6S (4.7") & 6S Plus (5.5"), Black',
                       'Home Button Flex Cable for the iPhone 6S (4.7") & 6S Plus (5.5"), Silver',
                       'Home Button Flex Cable for the iPhone 6S (4.7") & 6S Plus (5.5"), Gold',
                       'Home Button Flex Cable for the iPhone 6S (4.7") & 6S Plus (5.5"), Rose Gold']

# iPhone 6S
iPhone6S_Home = HQPricing('iPhone 6S', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [46]:
sentrix_descriptions = ['Home Button With Flex For iPhone 5S / SE (Gold)',
                        'Home Button With Flex For iPhone 5S / SE (Silver)'
                        'Home Button With Flex For iPhone 5S / SE (Space Grey)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 5S / iPhone SE - Black (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 5S / iPhone SE - Gold (Fingerprint scanner is aftermarket - Biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 5S / iPhone SE - White (Fingerprint scanner is aftermarket - Biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPhone SE, Black',
                       'Home Button Flex Cable for the iPhone 5E, Silver',
                       'Home Button Flex Cable for the iPhone 5E, Gold',
                       'Home Button Flex Cable for the iPhone SE, Rose Gold']

# iPhone SE
iPhoneSE_Home = HQPricing('iPhone SE', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [47]:
sentrix_descriptions = ['LCD Assembly for iPhone 5C (Aftermarket Quality, Shenchao) (White)',
                        'LCD Assembly for iPhone 5C (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly for iPhone 5C (Aftermarket Plus Quality, Tianma) (White)',
                        'LCD Assembly for iPhone 5C (Aftermarket Plus Quality, Tianma) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 5C (Select) - Black', 
                         'LCD & Digitizer Frame Assembly for iPhone 5C (Select) - White']

gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 5C',
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 5C',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 5C',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 5C']

# iPhone 5C
iPhone5C_Screen = HQPricing('iPhone 5C', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [48]:
sentrix_descriptions = ['LCD Assembly for iPhone 5S/SE (Aftermarket Quality, Shenchao) (White)',
                        'LCD Assembly for iPhone 5S/SE (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly for iPhone 5S / SE (Aftermarket Plus Quality, Tianma) (White)',
                        'LCD Assembly for iPhone 5S / SE (Aftermarket Plus Quality, Tianma) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 5S (Select) - Black', 
                         'LCD & Digitizer Frame Assembly for iPhone 5S (Select) - White']

gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 5S',
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 5S',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 5S',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 5S']

# iPhone 5S
iPhone5S_Screen = HQPricing('iPhone 5S', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [49]:
sentrix_descriptions = ['LCD Assembly For iPhone 6 Plus (Aftermarket Quality, HuaXing) (White)',
                        'LCD Assembly For iPhone 6 Plus (Aftermarket Quality, HuaXing) (Black)',
                        'LCD Assembly For iPhone 6 Plus (Aftermarket Plus Quality, XO4) (White)',
                        'LCD Assembly For iPhone 6 Plus (Aftermarket Plus Quality, XO4) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 6 Plus (5.5") (Select) - White (Bulk Pricing Available)', 
                        'LCD & Digitizer Frame Assembly for iPhone 6 Plus (5.5") (Basic) - White (Bulk Pricing Available)',
                        'LCD & Digitizer Frame Assembly for iPhone 6 Plus (5.5") (Select) - Black (Bulk Pricing Available)',
                        'LCD & Digitizer Frame Assembly for iPhone 6 Plus (5.5") (Basic) - Black (Bulk Pricing Available)']


gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 6 Plus (5.5")',
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 6 Plus (5.5")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 6 Plus (5.5")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 6 Plus (5.5")']

# iPhone 6 Plus
iPhone6Plus_Screen = HQPricing('iPhone 6 Plus', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [50]:
sentrix_descriptions = ['LCD Assembly For iPhone 6S Plus (Aftermarket Quality, HuaXing) (White)',
                        'LCD Assembly For iPhone 6S Plus (Aftermarket Quality, HuaXing) (Black)',
                        'LCD Assembly For iPhone 6S Plus (Aftermarket Plus Quality, XO4) (White)',
                        'LCD Assembly For iPhone 6S Plus (Aftermarket Plus Quality, XO4) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 6S Plus (5.5") (Select) - White (Bulk Pricing Available)', 
                        'LCD & Digitizer Frame Assembly for iPhone 6S Plus (5.5") (Basic) - White (Bulk Pricing Available)',
                        'LCD & Digitizer Frame Assembly for iPhone 6S Plus (5.5") (Select) - Black (Bulk Pricing Available)',
                        'LCD & Digitizer Frame Assembly for iPhone 6S Plus (5.5") (Basic) - Black (Bulk Pricing Available)']


gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly for iPhone 6S Plus (5.5") , White',
                       'Aftermarket LCD Screen and Digitizer Assembly for iPhone 6S Plus (5.5") , Black',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (AUO LCD), White, for iPhone 6S Plus (5.5")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (AUO LCD), Black, for iPhone 6S Plus (5.5")' ]

# iPhone 6S Plus
iPhone6SPlus_Screen = HQPricing('iPhone 6S Plus', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [51]:
sentrix_descriptions = ['Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Silver) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Silver) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Rose Gold) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Jet Black / Matte Black) (Warning: For Cosmetic Use Only)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 7 (4.7") - Black (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 7 (4.7") - White (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 7 (4.7") - Rose Gold (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 7 (4.7") - Gold (Cosmetic replacement - button will not function after repair)']

gadget_descriptions = ['Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Black (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, White (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Gold (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Rose Gold (Cosmetic only).']

# iPhone 7
iPhone7_Home = HQPricing('iPhone 7', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [52]:
sentrix_descriptions = ['Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Silver) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Silver) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Rose Gold) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Jet Black / Matte Black) (Warning: For Cosmetic Use Only)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 7 Plus (5.5") - Black (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 7 Plus (5.5") - White (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 7 Plus (5.5") - Rose Gold (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 7 Plus (5.5") - Gold (Cosmetic replacement - button will not function after repair)']


gadget_descriptions = ['Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Black (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, White (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Gold (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Rose Gold (Cosmetic only).']

# iPhone 7 Plus
iPhone7Plus_Home = HQPricing('iPhone 7 Plus', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [53]:
sentrix_descriptions = ['LCD For iPad 2 (Premium)']

defender_descriptions = ['LCD & Digitizer for iPad Air 2 (Prime) - Black',
                         'LCD & Digitizer for iPad Air 2 (Prime) - White']


gadget_descriptions = ['Premium Glass and Digitizer Full Assembly LCD, White, for iPad Air 2 (Sleep/Wake Flex Pre-Soldered)',
                       'Premium Glass and Digitizer Full Assembly LCD, Black, for iPad Air 2 (Sleep/Wake Flex Pre-Soldered)']

# iPad Air 2
iPadAir2_Screen = HQPricing('iPad Air 2', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [54]:
sentrix_descriptions = ['Charging Port Flex Cable For iPad 2']

defender_descriptions = ['Charging Port Flex Cable for iPad Air 2 - White',
                         'Charging Port Flex Cable for iPad Air 2 - Black']

gadget_descriptions = ['Dock Connector Charging Port for iPad Air 2 (Black)',
                       'Dock Connector Charging Port for iPad Air 2 (White)']

# iPad Air 2
iPadAir2_charging = HQPricing('iPad Air 2', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['']

defender_descriptions = ['Home Button (w/ Fingerprint Scanner) for iPad Air 2 - Black (Fingerprint scanner is aftermarket - biometrics will not work)',
                         'Home Button (w/ Fingerprint Scanner) for iPad Air 2 - Gold (Fingerprint scanner is aftermarket - biometrics will not work)',
                         'Home Button (w/ Fingerprint Scanner) for iPad Air 2 - Silver (Fingerprint scanner is aftermarket - biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPad Air 2, Black'
                       'Home Button Flex Cable for the iPad Air 2, Silver',
                       'Home Button Flex Cable for the iPad Air 2, Gold']

# iPad Air 2
iPadAir2_Home = HQPricing('iPad Air 2', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [55]:
sentrix_descriptions = ['LCD For iPad Air 1 / iPad 5 (Premium)']

defender_descriptions = ['LCD for iPad Air / iPad 5 (2017)']


gadget_descriptions = ['LCD Display Panel Monitor Screen for iPad Air / iPad 5 2017 Model']

# iPad 5
iPad5_Screen = HQPricing('iPad5', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Charging Port Flex Cable For iPad Air 1 / iPad 5 (2017) / iPad 6 (2018) (White)',
                        'Charging Port Flex Cable For iPad Air 1 / iPad 5 (2017) / iPad 6 (2018) (Black)']

defender_descriptions = ['Charging Port Flex Cable for iPad Air / iPad 5 (2017) / iPad 6 (2018) - Black',
                         'Charging Port Flex Cable for iPad Air / iPad 5 (2017) / iPad 6 (2018) - White']

gadget_descriptions = ['Dock Connector Charging Port for iPad Air / iPad 5 2017 / iPad 6 2018 (Black)',
                       'Dock Connector Charging Port for iPad Air / iPad 5 2017 / iPad 6 2018 (White)']

# iPad5
iPad5_charging = HQPricing('iPad5', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Home Button Flex Cable For iPad 5 (2017) / iPad 6 (2018) (White)',
                        'Home Button Flex Cable For iPad 5 (2017) / iPad 6 (2018) (Black)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPad 5 (2017) / iPad 6 (2018) - Space Gray (Fingerprint scanner is aftermarket - biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPad 5 (2017) / iPad 6 (2018) - Silver (Fingerprint scanner is aftermarket - biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPad 5 (2017) / iPad 6 (2018) - Gold (Fingerprint scanner is aftermarket - biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPad 5 (2017) / iPad 6 (2018) , Black',
                       'Home Button Flex Cable for the iPad 5 (2017) / iPad 6 (2018) , White']

# iPad5
iPad5_Home = HQPricing('iPad5', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [56]:
sentrix_descriptions = ['Digitizer With Home Button Flex For iPad Air 1 / iPad 5 (2017) (Premium Quality) (White)',
                        'Digitizer With Home Button Flex For iPad Air 1 / iPad 5 (2017) (Premium Quality) (Black)']

defender_descriptions = ['Digitizer for iPad Air / iPad 5 (Prime) - Black',
                         'Digitizer for iPad Air / iPad 5 (Prime) - White']

gadget_descriptions = ['Premium Black Touch Screen Glass Digitizer for iPad 5 (2017)',
                       'Premium Black Touch Screen Glass Digitizer for iPad 5 (2017)']

# iPad 5
iPad5_Digitizer = HQPricing('iPad5', 'Digitizer', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Digitizer For iPad 6 (2018) (Premium) (White)',
                        'Digitizer For iPad 6 (2018) (Premium) (Black)']

defender_descriptions = ['Digitizer for iPad 6 (A1893 / A1954) (9.7") (Prime) - Black',
                         'Digitizer for iPad 6 (A1893 / A1954) (9.7") (Prime) - White']

gadget_descriptions = ['Premium White Touch Screen Glass Digitizer for iPad 6 (2018)',
                       'Premium Black Touch Screen Glass Digitizer for iPad 6 (2018)']

# iPad 6
iPad6_Digitizer = HQPricing('iPad6', 'Digitizer', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Replacement Battery For iPad Air 1 / iPad 5 (2017) / iPad 6 (2018) (Premium)']

defender_descriptions = ['Battery for iPad Air / iPad 5 (2017) / iPad 6 (2018)']

gadget_descriptions = ['Battery for the iPad Air 1 1st Generation / iPad 5 (2017) / iPad 6 (2018)']

# iPad 5
iPad5_Battery = HQPricing('iPad5', 'Battery', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

# iPad 6
iPad6_Battery = HQPricing('iPad6', 'Battery', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['LCD For iPad 6 (2018) (Premium)']

defender_descriptions = ['LCD for iPad 6 (A1893 / A1954) (9.7") (2018)']

gadget_descriptions = ['LCD Screen for iPad 6 (2018)']

# iPad 6
iPad6_Screen = HQPricing('iPad6', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Charging Port Flex Cable For iPad Air 1 / iPad 5 (2017) / iPad 6 (2018) (White)',
                        'Charging Port Flex Cable For iPad Air 1 / iPad 5 (2017) / iPad 6 (2018) (Black)']

defender_descriptions = ['Charging Port Flex Cable for iPad Air / iPad 5 (2017) / iPad 6 (2018) - Black',
                         'Charging Port Flex Cable for iPad Air / iPad 5 (2017) / iPad 6 (2018) - White']

gadget_descriptions = ['Dock Connector Charging Port for iPad Air / iPad 5 2017 / iPad 6 2018 (Black)',
                       'Dock Connector Charging Port for iPad Air / iPad 5 2017 / iPad 6 2018 (White)']

# iPad 6
iPad6_charging = HQPricing('iPad6', 'Charging Port', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

sentrix_descriptions = ['Home Button Flex Cable For iPad 5 (2017) / iPad 6 (2018) (White)',
                        'Home Button Flex Cable For iPad 5 (2017) / iPad 6 (2018) (Black)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPad 5 (2017) / iPad 6 (2018) - Space Gray (Fingerprint scanner is aftermarket - biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPad 5 (2017) / iPad 6 (2018) - Silver (Fingerprint scanner is aftermarket - biometrics will not work)',
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPad 5 (2017) / iPad 6 (2018) - Gold (Fingerprint scanner is aftermarket - biometrics will not work)']

gadget_descriptions = ['Home Button Flex Cable for the iPad 5 (2017) / iPad 6 (2018) , Black',
                       'Home Button Flex Cable for the iPad 5 (2017) / iPad 6 (2018) , White']

# iPad 6
iPad6_Home = HQPricing('iPad6', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [57]:
sentrix_descriptions = ['Replacement battery For iPad 2 (Premium)']

defender_descriptions = ['Battery for iPad Air 2']


gadget_descriptions = ['Battery for the iPad Air 2 2nd Generation']

# iPad Air 2
iPadAir2_Battery = HQPricing('iPad Air 2', 'Battery', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [58]:
sentrix_descriptions = ['Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Silver) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Silver) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Rose Gold) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Jet Black / Matte Black) (Warning: For Cosmetic Use Only)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 8 Plus (5.5") - Black (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 8 Plus (5.5") - White (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 8 Plus (5.5") - Rose Gold (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 8 Plus (5.5") - Gold (Cosmetic replacement - button will not function after repair)']


gadget_descriptions = ['Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Black (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, White (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Gold (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Rose Gold (Cosmetic only).']

# iPhone 8 Plus
iPhone8Plus_Home = HQPricing('iPhone 8 Plus', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [59]:
sentrix_descriptions = ['Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Silver) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Silver) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Rose Gold) (Warning: For Cosmetic Use Only)',
                        'Home Button With Flex For iPhone 7 / 7 Plus / 8 / 8 Plus (Jet Black / Matte Black) (Warning: For Cosmetic Use Only)']

defender_descriptions = ['Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 8 (4.7") - Black (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 8 (4.7") - White (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 8 (4.7") - Rose Gold (Cosmetic replacement - button will not function after repair)', 
                         'Home Button Flex Cable (w/ Fingerprint Scanner) for iPhone 8 (4.7") - Gold (Cosmetic replacement - button will not function after repair)']

gadget_descriptions = ['Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Black (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, White (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Gold (Cosmetic only).',
                       'Home Button Flex Cable for the iPhone 7 / 7 Plus / 8 / 8 Plus, Rose Gold (Cosmetic only).']

# iPhone 8 
iPhone8_Home = HQPricing('iPhone 8', 'Home', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

In [60]:
sentrix_descriptions = ['LCD Assembly For iPhone 6S (Aftermarket Quality, Shenchao) (White)',
                        'LCD Assembly For iPhone 6S (Aftermarket Quality, Shenchao) (Black)',
                        'LCD Assembly For iPhone 6S (Aftermarket Plus Quality, XO4) (White)',
                        'LCD Assembly For iPhone 6S (Aftermarket Plus Quality, XO4) (Black)']

defender_descriptions = ['LCD & Digitizer Frame Assembly for iPhone 6S (4.7") (Select) - White (Bulk Pricing Available)', 
                        'LCD & Digitizer Frame Assembly for iPhone 6S (4.7") (Basic) - White (Bulk Pricing Available)',
                        'LCD & Digitizer Frame Assembly for iPhone 6S (4.7") (Select) - Black (Bulk Pricing Available)',
                        'LCD & Digitizer Frame Assembly for iPhone 6S (4.7") (Basic) - Black (Bulk Pricing Available)']


gadget_descriptions = ['Aftermarket LCD Screen and Digitizer Assembly, White, for iPhone 6S (4.7")',
                       'Aftermarket LCD Screen and Digitizer Assembly, Black, for iPhone 6S (4.7")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), White, for iPhone 6S (4.7")',
                       'Midgrade LCD Panel Screen and Digitizer Assembly (Tianma LCD), Black, for iPhone 6S (4.7")']

# iPhone 6S
iPhone6S_Screen = HQPricing('iPhone 6S', 'Screen', screenPrices(df_defender, defender_descriptions),
                           screenPrices(df_gadgets, gadget_descriptions),
                           screenPrices(df_sentrix, sentrix_descriptions))

# iPad 5
iPad5_Display = HQPricing('iPad5', 'Display', round(iPad5_Digitizer.defender + iPad5_Screen.defender, 2), 
                          round(iPad5_Digitizer.gadgets + iPad5_Screen.gadgets, 2), 
                          round(iPad5_Digitizer.sentrix + iPad5_Screen.sentrix, 2))

# iPad 6
iPad6_Display = HQPricing('iPad6', 'Display', round(iPad6_Digitizer.defender + iPad6_Screen.defender, 2), 
                          round(iPad6_Digitizer.gadgets + iPad6_Screen.gadgets, 2), 
                          round(iPad6_Digitizer.sentrix + iPad6_Screen.sentrix, 2))

In [61]:
phones = [iPhoneX, iPhone8, iPhone7, iPhone6, iPhone5, iPhone5C, iPhone5S, iPhone6S, iPhoneSE, 
          iPhone7Plus, iPhone6Plus, iPhone6SPlus, iPhone8Plus, iPhone5_Screen, iPhone5C_Screen,
          iPhone5S_Screen, iPhone6_Screen, iPhone6Plus_Screen, iPhone6S_Screen, iPhone6SPlus_Screen, 
          iPhone7_Screen, iPhone8_Screen, iPhone7Plus_Screen, iPhone8Plus_Screen, iPhoneSE_Screen,
          iPhoneX_Screen, iPhone7Plus_charging, iPhone7_charging, iPhone8_charging, iPhone8Plus_charging,
          iPhone6SPlus_charging, iPhone6S_charging, iPhoneX_charging, iPhoneSE_charging, iPhone5_charging,
          iPhone5C_charging, iPhone5S_charging, iPhone6_charging, iPhone6Plus_charging, iPhone5_Home,
          iPhone5C_Home, iPhone5S_Home, iPhoneSE_Home, iPhone6S_Home, iPhone6SPlus_Home, iPhone6Plus_Home,
          iPhone6_Home, iPhone7_Home, iPhone7Plus_Home, iPhone8Plus_Home, iPhone8_Home,
          iPadAir2_Screen, iPadAir2_Battery, iPadAir2_Home, iPadAir2_charging, iPad5_Home, iPad5_charging,
          iPad5_Screen, iPad6_Screen, iPad6_charging, iPad6_Home, iPad5_Battery, iPad6_Battery,
          iPad5_Digitizer, iPad6_Digitizer, iPad5_Display, iPad6_Display, iPhoneXR_Screen, iPhoneXR_charging,
          iPhoneXS_Screen, iPhoneXS_charging, SamsungNote9_Screen, SamsungNote9_Battery, SamsungNote9_charging,
          SamsungGS9_Screen, SamsungGS9_Battery]

In [62]:
layouts = []
            
for phone in phones:
    layouts.append([phone.product, phone.part, phone.defender, phone.gadgets, phone.sentrix])
    
df_phone_parts = pd.DataFrame(layouts, columns = ['Product', 'Part', 'Defender', 'Gadgets', 'Sentrix'])

df_phone_parts[['Defender','Gadgets', 'Sentrix']] = df_phone_parts[['Defender','Gadgets', 'Sentrix']].apply(pd.to_numeric, errors = 'ignore')

df_phone_parts['Average'] = 0.0
df_phone_parts['Median'] = 0.0
df_phone_parts['Offer'] = 0.0

df_phone_parts['Average'] = df_phone_parts.apply(lambda x: Average([x.Defender, x.Gadgets, x.Sentrix]), axis = 1)
df_phone_parts['Median'] = df_phone_parts.apply(lambda x: Median([x.Defender, x.Gadgets, x.Sentrix]), axis = 1)
df_phone_parts['Offer'] = df_phone_parts.apply(lambda x: np.minimum(x.Average, x.Median), axis = 1)
df_phone_parts['Savings'] = df_phone_parts.apply(lambda x: x.Average - x.Offer, axis = 1)
    
df_phone_parts.sort_values(by = ['Product', 'Part'], ascending = [True, True], inplace = True)
df_phone_parts.reset_index(drop = True, inplace = True)
df_phone_parts.fillna(0.0, inplace = True)
df_phone_parts.replace(0.0, '', inplace = True)
# df_phone_parts.Part = df_phone_parts.Part.astype("category")
# df_phone_parts.sort_values(['Part'])
df_phone_parts.Part = pd.Categorical(df_phone_parts['Part'], ["Digitizer", "Screen", "Display", "Battery", "Home", "Charging Port"])
df_phone_parts.sort_values(by = ['Product', 'Part'], ascending = [True, True], inplace = True)
df_phone_parts.reset_index(drop = True, inplace = True)
df_phone_parts

,Product,Part,Defender,Gadgets,Sentrix,Average,Median,Offer,Savings
0,Samsung Galaxy S9,Screen,203.24,259,202.38,221.54,203.24,203.24,18.3
1,Samsung Galaxy S9,Battery,6.99,,11.51,9.25,9.25,9.25,
2,Samsung Note 9,Screen,241.99,259,238.03,246.34,241.99,241.99,4.35
3,Samsung Note 9,Battery,7.99,10.03,,9.01,9.01,9.01,
4,Samsung Note 9,Charging Port,,7.88,3.02,5.45,5.45,5.45,
5,iPad Air 2,Screen,,119,39,79.00,79.00,79.00,
6,iPad Air 2,Battery,17.99,14.29,9.64,13.97,14.29,13.97,
7,iPad Air 2,Home,1.99,1.66,,1.82,1.82,1.82,
8,iPad Air 2,Charging Port,2.99,1.43,1.24,1.89,1.43,1.43,0.46
9,iPad5,Digitizer,,11.56,14.82,13.19,13.19,13.19,


In [65]:
writer = pd.ExcelWriter('HQ Pricing.xlsx')
df_phone_parts.to_excel(writer, 'HQ Pricing', index = False)
wb = writer.book
ws = writer.sheets['HQ Pricing']
money_fmt = wb.add_format({'num_format': '$#,##.00'})
ws.set_column('C:I', 12, money_fmt)
writer.save()

In [72]:
if sys.platform == 'win32':
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    print('The output file is located in the following directory: ' + os.getcwd())
    filename = os.getcwd() + '\\HQ Pricing.xlsx'
    wb = excel.Workbooks.Open(filename)
    ws = wb.Worksheets("HQ Pricing")
    ws.Columns.AutoFit()
    wb.Save()
    excel.Application.Quit()

The output file is located in the following directory: C:\Users\DYancy
